In [2]:
import pandas as pd
import random
import time

# Load data
courses_df = pd.read_csv('/content/Courses.csv')
teachers_df = pd.read_csv('/content/NewTeachers.csv')
rooms_df = pd.read_csv('/content/newrooms.csv')
slots_df = pd.read_csv('/content/SLOTS.csv')
sections_df = pd.read_csv('/content/SECTION (1).csv')
constraints_df = pd.read_csv('/content/constraints.csv')

# Parse the new time slots
def parse_time_slots(slots_df):
    time_slots = []
    for day_index, row in slots_df.iterrows():
        day = row['Day']
        for slot_index, slot in enumerate(slots_df.columns[1:]):
            time_slots.append(f"{day}_{slot_index + 1}")
    return time_slots

time_slot_ids = parse_time_slots(slots_df)



# Parameters
POPULATION_SIZE = 150
GENERATIONS = 200
MUTATION_RATE = 0.3
section_column = 'section'

# Fitness function
def fitness(individual, section_ids, course_class_requirements, MAX_HOURS_PER_DAY, MAX_HOURS_PER_WEEK, MIN_TIME_BETWEEN_CLASSES):
    conflicts = 0
    course_counts = {}
    daily_hours = {}
    weekly_hours = {}
    section_slots = {}
    teacher_slots = {}

    for class_ in individual:
        section_id = class_["New Section"]
        course_id = class_["Course_ID"]
        teacher_id = class_["Teacher_ID"]
        slot_id = class_["Slot_ID"]
        day, time = slot_id.split('_')

        # Track counts
        key = (section_id, course_id)
        course_counts[key] = course_counts.get(key, 0) + 1

        # Track daily and weekly hours
        daily_hours.setdefault(section_id, {}).setdefault(day, 0)
        daily_hours.setdefault(teacher_id, {}).setdefault(day, 0)
        weekly_hours[section_id] = weekly_hours.get(section_id, 0) + 1
        weekly_hours[teacher_id] = weekly_hours.get(teacher_id, 0) + 1

        daily_hours[section_id][day] += 1
        daily_hours[teacher_id][day] += 1

        # Track slots
        section_slots.setdefault(section_id, []).append(slot_id)
        teacher_slots.setdefault(teacher_id, []).append(slot_id)

    # Check daily and weekly limits
    for section_id in section_ids:
        for day_hours in daily_hours.get(section_id, {}).values():
            if day_hours > MAX_HOURS_PER_DAY:
                conflicts += (day_hours - MAX_HOURS_PER_DAY) * 10
        if weekly_hours.get(section_id, 0) > MAX_HOURS_PER_WEEK:
            conflicts += (weekly_hours[section_id] - MAX_HOURS_PER_WEEK) * 10

    for teacher_id in teacher_slots:
        for day_hours in daily_hours.get(teacher_id, {}).values():
            if day_hours > MAX_HOURS_PER_DAY:
                conflicts += (day_hours - MAX_HOURS_PER_DAY) * 10
        if weekly_hours.get(teacher_id, 0) > MAX_HOURS_PER_WEEK:
            conflicts += (weekly_hours[teacher_id] - MAX_HOURS_PER_WEEK) * 10

    # Check minimum time between classes
    for slots in section_slots.values():
        sorted_slots = sorted(slots)
        for i in range(1, len(sorted_slots)):
            time_diff = abs(int(sorted_slots[i].split('_')[1]) - int(sorted_slots[i - 1].split('_')[1]))
            if time_diff < MIN_TIME_BETWEEN_CLASSES:
                conflicts += 5

    for slots in teacher_slots.values():
        sorted_slots = sorted(slots)
        for i in range(1, len(sorted_slots)):
            time_diff = abs(int(sorted_slots[i].split('_')[1]) - int(sorted_slots[i - 1].split('_')[1]))
            if time_diff < MIN_TIME_BETWEEN_CLASSES:
                conflicts += 5

    # Penalize for unmet or excess course requirements
    for section_id in section_ids:
        for course_id, required_count in course_class_requirements.items():
            count = course_counts.get((section_id, course_id), 0)
            if count != required_count:
                conflicts += abs(count - required_count) * 10

    return conflicts

# Crossover function
def crossover(parent1, parent2):
    child = []
    for gene1, gene2 in zip(parent1, parent2):
        child.append(gene1 if random.random() < 0.5 else gene2)
    return child

# Mutation function
def mutate(individual, rooms_df, teachers_df, time_slot_ids):
    class_to_mutate = random.choice(individual)

    # Mutate Room_ID
    class_to_mutate["Room_ID"] = random.choice(rooms_df["Room_ID"].tolist())

    # Mutate Slot_ID
    class_to_mutate["Slot_ID"] = random.choice(time_slot_ids)

    # Mutate Teacher_ID
    class_to_mutate["Teacher_ID"] = random.choice(teachers_df["Teacher_ID"].tolist())



def initialize_population(section_ids, course_class_requirements, teachers_df, rooms_df, time_slot_ids):
    population = []
    for _ in range(POPULATION_SIZE):
        individual = []
        for section_id in section_ids:
            course_counts = {course_id: 0 for course_id in course_class_requirements.keys()}
            for course_id, required_count in course_class_requirements.items():
                while course_counts[course_id] < required_count:
                    teacher_id = random.choice(teachers_df["Teacher_ID"].tolist())
                    room_id = random.choice(rooms_df["Room_ID"].tolist())
                    slot_id = random.choice(time_slot_ids)
                    individual.append({
                        "New Section": section_id,
                        "Course_ID": course_id,
                        "Teacher_ID": teacher_id,
                        "Room_ID": room_id,
                        "Slot_ID": slot_id
                    })
                    course_counts[course_id] += 1
        population.append(individual)
    return population



def genetic_algorithm_per_year(section_ids, course_class_requirements, teachers_df, rooms_df, time_slot_ids, constraints_df, MAX_HOURS_PER_DAY=8, MAX_HOURS_PER_WEEK=32, MIN_TIME_BETWEEN_CLASSES=1, generations=GENERATIONS):
    population = initialize_population(section_ids, course_class_requirements, teachers_df, rooms_df, time_slot_ids)
    best_individual = None
    best_fitness = float('inf')

    for generation in range(generations):
        # Evaluate fitness
        fitness_scores = [(ind, fitness(ind, section_ids, course_class_requirements, MAX_HOURS_PER_DAY, MAX_HOURS_PER_WEEK, MIN_TIME_BETWEEN_CLASSES)) for ind in population]
        fitness_scores.sort(key=lambda x: x[1])

        # Update best individual
        if fitness_scores[0][1] < best_fitness:
            best_individual = fitness_scores[0][0]
            best_fitness = fitness_scores[0][1]

        # Selection: Select top 50% as parents
        parents = [ind for ind, _ in fitness_scores[:POPULATION_SIZE // 2]]

        # Generate offspring
        offspring = []
        while len(offspring) < POPULATION_SIZE:
            parent1, parent2 = random.sample(parents, 2)
            child = crossover(parent1, parent2)
            if random.random() < MUTATION_RATE:
                mutate(child, rooms_df, teachers_df, time_slot_ids)
            offspring.append(child)

        population = offspring

    return best_individual, best_fitness



def display_timetable_by_section(schedule):
    # Create a DataFrame from the schedule for easier manipulation
    timetable = pd.DataFrame(schedule)

    # Define the days and slots structure
    days_of_week = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THRUSDAY', 'FRIDAY']
    slots_per_day = ['SLOT-1', 'SLOT-2', 'SLOT-3', 'SLOT-4', 'SLOT-5', 'SLOT-6']

    for section_id in timetable["New Section"].unique():
        print(f"\nTimetable for Section {section_id}")

        # Create a dictionary to hold the schedule for each day
        section_schedule = {day: [''] * 6 for day in days_of_week}

        # Populate the schedule for the section
        for _, row in timetable[timetable["New Section"] == section_id].iterrows():
            course_id = row["Course_ID"]
            room_id = row["Room_ID"]
            slot_id = row["Slot_ID"]
            day, slot_num = slot_id.split('_')
            slot_num = int(slot_num) - 1  # Adjust slot index to start from 0

            # Add the course and room information
            section_schedule[day][slot_num] = f"{course_id} (Room {room_id})"

        # Print the formatted schedule
        print(f"{'Day':<10} {'SLOT-1':<20} {'SLOT-2':<20} {'SLOT-3':<20} {'SLOT-4':<20} {'SLOT-5':<20} {'SLOT-6':<20}")
        for day in days_of_week:
            print(f"{day:<10}", end=" ")
            for slot in range(6):
                print(f"{section_schedule[day][slot]:<20}", end=" ")
            print()  # New line after each day
def generate_timetable_for_selected_year(courses_df, sections_df, teachers_df, rooms_df, time_slot_ids, constraints_df):
    # Get available years from the sections DataFrame
    available_years = sections_df['year'].unique()
    print(f"Available Years: {', '.join(map(str, sorted(available_years)))}")

    # Prompt the user to select a year
    while True:
        try:
            selected_year = int(input("Enter the year for which you want to generate the timetable: "))
            if selected_year in available_years:
                break
            else:
                print(f"Year {selected_year} is not available. Please choose a valid year.")
        except ValueError:
            print("Invalid input. Please enter a valid year (numeric).")

    print(f"\n--- Generating Timetable for Year {selected_year} ---")

    # Filter sections for the selected year
    year_sections = sections_df[sections_df['year'] == selected_year]['section'].tolist()

    # Filter courses for the selected year
    year_courses_df = courses_df[courses_df['Year'] == selected_year]

    # Define course requirements for the selected year
    course_class_requirements = {
        course_id: row['Credits'] for course_id, row in year_courses_df[['Course_ID', 'Credits']].set_index('Course_ID').iterrows()
    }

    # Run the genetic algorithm for the selected year
    best_schedule, best_fit = genetic_algorithm_per_year(
        section_ids=year_sections,
        course_class_requirements=course_class_requirements,
        teachers_df=teachers_df,
        rooms_df=rooms_df,
        time_slot_ids=time_slot_ids,
        constraints_df=constraints_df
    )

    print(f"Best Fitness for Year {selected_year}: {best_fit}")

    # Save the timetable
    timetable_df = pd.DataFrame(best_schedule)
    timetable_file = f"timetable_year_{selected_year}.csv"
    timetable_df.to_csv(timetable_file, index=False)
    print(f"Timetable for Year {selected_year} saved to '{timetable_file}'")

    # Display the timetable
    display_timetable_by_section(best_schedule)

    return timetable_df


# Execute the function to generate the timetable for a selected year
start_time = time.time()
timetable_for_selected_year = generate_timetable_for_selected_year(
    courses_df=courses_df,
    sections_df=sections_df,
    teachers_df=teachers_df,
    rooms_df=rooms_df,
    time_slot_ids=time_slot_ids,
    constraints_df=constraints_df
)
print(f"\nTotal Execution Time: {time.time() - start_time:.2f} seconds")

Available Years: 1, 2, 3, 4
Enter the year for which you want to generate the timetable: 4

--- Generating Timetable for Year 4 ---
Best Fitness for Year 4: 630
Timetable for Year 4 saved to 'timetable_year_4.csv'

Timetable for Section A1
Day        SLOT-1               SLOT-2               SLOT-3               SLOT-4               SLOT-5               SLOT-6              
MONDAY     C410 (Room CR_203)                                                                                       C405 (Room CR_104)   
TUESDAY    C410 (Room CR_102)   C402 (Room CR_101)   C403 (Room CR_601)   C401 (Room CR_401)                        C402 (Room CR_102)   
WEDNESDAY                                                                 C408 (Room CR_103)                                             
THRUSDAY                                             C401 (Room CR_403)                        C406 (Room CR_503)   C403 (Room CR_606)   
FRIDAY                          C409 (Room CR_407)                     